<a href="https://colab.research.google.com/github/whoami-Lory271/DL-project/blob/main/TPTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install library import 

In [1]:
!pip install pytorch-lightning --quiet
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 27.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 13.3 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=8e8d930dce5a0b1da82aa82bd942b6f857871e564bf5318cf4fae0d990d5550f
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Att

In [4]:
!pip install --upgrade gdown --quiet

In [5]:

import gdown
from numpy import array
from sklearn.preprocessing import OneHotEncoder
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.normalization import LayerNorm
import pickle
import os
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from pathlib import Path
import pandas as pd
import numpy as np
import csv
import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning.callbacks.progress import TQDMProgressBar

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Helper Function

In [3]:
def get_padding_mask(X):
  pad = X == 0
  padding_mask = pad.repeat(1,1,X.shape[1]).reshape((X.shape[0],X.shape[1],X.shape[1]))
  padding_mask[pad] = True
  return padding_mask

In [6]:
def create_padding(encode, vocab, max_len):
    enc = np.array(encode)
    encoded = np.zeros(max_len + 2)
    for k in range(len(enc)):
      encoded[k] = enc[k]
    return encoded

#Dataset

Download what you need for the dataset:



In [7]:
url = "https://drive.google.com/drive/folders/1c0WtL7iRAhhGGsXL3YvOZpIye3TjPmY3?usp=share_link" #easy
gdown.download_folder(url = url, quiet = True, use_cookies=False)  

url = "https://drive.google.com/drive/folders/1c0pUsyGRDk-lbWjkaiI5lnIPyx-uo7os?usp=share_link" #medium
gdown.download_folder(url = url, quiet = True, use_cookies=False)

url = "https://drive.google.com/drive/folders/1c1IS1u2EsnJ1PjEfkqEn9ZeKO_jb9O76?usp=share_link" #hard
gdown.download_folder(url = url, quiet = True, use_cookies=False)

!gdown "1-7ies2HEUy56W8akeHlBN-XkPDPy9uh0" #vocab

Downloading...
From: https://drive.google.com/uc?id=1-7ies2HEUy56W8akeHlBN-XkPDPy9uh0
To: /content/vocabs
100% 376/376 [00:00<00:00, 942kB/s]


In [8]:
class MathematicsDataset(Dataset):
  def __init__(self, train, max_len_quest=160, max_len_answ=30):
    self.max_len_quest = max_len_quest
    self.max_len_answ = max_len_answ
    #unpickle the vocabulary
    with open(os.path.join("/content",'vocabs'),'rb') as infile:
      self.vocab = pickle.load(infile)

    train_type = ["easy","medium", "hard"]
    assert train in train_type, f"the type should be easy,medium or hard!" 
    self.path = "/content/"+ train + "_pickle"

    #unpickle the qestions
    with open(os.path.join(self.path,"questions"),'rb') as infile:
      self.quest = pickle.load(infile)

    #unpickle the answers
    with open(os.path.join(self.path,"answers"),'rb') as infile:
      self.answ = pickle.load(infile)
  
  def __len__(self):
    return len(self.file_quest)

  def __getitem__(self, idx):
    assert(idx  < len(self.quest)) #indices should start from 0 to len - 1 (there are 5999994 elements)
    question = self.quest[idx]["Encoding"]
    encoding1 = torch.from_numpy(create_padding(question, self.vocab, self.max_len_quest))
    encoding1 = encoding1.type(torch.int)
    #self.sample["Question"] = encoding
      
    answer = self.answ[idx]["Encoding"]
    encoding2 = torch.from_numpy(create_padding(answer, self.vocab, self.max_len_answ))
    encoding2 = encoding2.type(torch.int)
    #self.sample["Answer"] = encoding
      
    return encoding1, encoding2 

In [ ]:
class PlMathematicsDataset(pl.LightningDataModule):
    def __init__(self, train_quest, train_answ, vocab , test_quest, test_answ, max_len_quest=160, max_len_answ=30, batch_size=512):
        super().__init__()
        self.train_quest = train_quest
        self.train_answ = train_answ
        self.test_quest = test_quest
        self.test_answ = test_answ
        self.vocab = vocab
        self.max_len_quest = max_len_quest
        self.max_len_answ = max_len_answ
        self.batch_size = batch_size
        self.dataset = MathematicsDataset(self.train_quest, self.train_answ, self.vocab)

    def prepare_data(self):
        # tok, load, ecc...
        # MNIST(self.data_dir, train=True, download=True)
        # MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):

        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            self.train_ds = MathematicsDataset(self.train_quest, self.train_answ, self.vocab)

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.test_ds = MathematicsDataset(self.test_quest, self.test_answ, self.vocab)

        if stage == "predict" or stage is None:
            self.predict_ds = MathematicsDataset(self.test_quest, self.test_answ, self.vocab)

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, shuffle = True, num_workers=12)

    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size=self.batch_size)

    def predict_dataloader(self):
        return DataLoader(self.predict_ds, batch_size=self.batch_size)

In [ ]:
#Create dataset
dataset = PlMathematicsDataset(quest, answ, vocab, test_quest, test_answ)

In [ ]:
# #Create dataloader
# train_loader = DataLoader(dataset, 32, shuffle=True)
# print(len(train_loader))

In [ ]:
# print("data", next(iter(train_loader)))
# print("questions_shape ", next(iter(train_loader))[0].shape)

# print("answers_shape ", next(iter(train_loader))[1].shape)

# Model

In [ ]:
class EmbeddingLayer(nn.Module):
  def __init__(self,vocabulary_size,embedding_dim):
    super().__init__()
    self.E = nn.Embedding(vocabulary_size,embedding_dim)
  
  def forward(self,x):
    return self.E(x)

In [ ]:
def position_embedding(batch_size,seq_length,emb_dim,pad_mask):
  res = torch.zeros((batch_size,seq_length,emb_dim),dtype=torch.float32)
  for pos in range(seq_length):
    for i in range(emb_dim):
      if i%2 == 0:
        res[:,pos,i] = math.sin(pos/10000**(2*i/emb_dim))
      else:
        res[:,pos,i] = math.cos(pos/10000**(2*i/emb_dim))
  res[pad_mask[:,0,:]] = 0
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  return res.to(device)

In [ ]:
class TPSelfAttention(nn.Module):
  def __init__(self,dz,dk):
    super().__init__()
    self.dk = dk
    self.Wq = nn.Linear(dz,dk)
    self.Wk = nn.Linear(dz,dk)
    self.Wv = nn.Linear(dz,dk)
    self.Wr = nn.Linear(dz,dk)
    self.Wo = nn.Linear(dk,dz)
    self.softmax = nn.Softmax(dim=2)
  
  def forward(self,x,padding_mask,enc=None,mask=False,other_mask=None):
    q = self.Wq(x)
    r = self.Wr(x)
    if enc == None:
      k = self.Wk(x)
      v = self.Wv(x)
    else:
      k = self.Wk(enc)
      v = self.Wv(enc)
    sc = torch.matmul(q,k.permute(0,2,1)) / math.sqrt(self.dk)

    if other_mask == None:
      sc[padding_mask] = float('-inf')
    else:
      qmod = q.clone()
      kmod = k.clone()
      qmod[padding_mask[:,0,:]] = 0
      kmod[other_mask[:,0,:]] = 0
      sc = torch.matmul(qmod,kmod.permute(0,2,1)) / math.sqrt(self.dk)
      sc[sc == 0] = float('-inf')

    if mask==True:
      for i in range(sc.shape[1]):
        sc[:,i,i+1:] = float('-inf')
    score = torch.matmul(torch.nan_to_num(self.softmax(sc)),v)
    return self.Wo(r * score)

In [ ]:
class TPMultiHeadAttention(nn.Module):
  def __init__(self,dmodel,nhead,dropout=0.1):
    super().__init__()
    self.nhead = nhead
    self.att_layers = nn.ModuleList([TPSelfAttention(dmodel,dmodel // nhead) for i in range(nhead)])
    #self.drop = nn.Dropout(p=dropout)
  
  def forward(self,x,padding_mask,enc=None,mask=False,other_mask=None):
    y = self.att_layers[0](x,padding_mask,enc=enc,mask=mask,other_mask=other_mask)
    for i in range(1,self.nhead):
      y += self.att_layers[i](x,padding_mask,enc=enc,mask=mask,other_mask=other_mask)
    #y = self.drop(y)
    return y

In [ ]:
class FFN(nn.Module):
  def __init__(self,dmodel,df,dropout=0.1):
    super().__init__()
    self.W1 = nn.Linear(dmodel,df)
    self.W2 = nn.Linear(df,dmodel)
    #self.drop = nn.Dropout(p=dropout)
  
  def forward(self,x):
    x = self.W1(x)
    x = F.relu(x)
    x = self.W2(x)
    #x = self.drop(x)
    return x

In [ ]:
class Encoder(nn.Module):
  def __init__(self,dmodel,df,nhead):
    super().__init__()
    self.norm = LayerNorm(dmodel)
    self.mha = TPMultiHeadAttention(dmodel,nhead)
    self.norm1 = LayerNorm(dmodel)
    self.ffn = FFN(dmodel,df)
    self.norm2 = LayerNorm(dmodel)
  
  def forward(self,x,padding_mask):
    x = self.norm(x)
    z = self.mha(x,padding_mask)
    z = self.norm1(x+z)
    y = self.ffn(z)
    return self.norm2(z+y)

In [ ]:
class Decoder(nn.Module):
  def __init__(self,dmodel,df,nhead):
    super().__init__()
    self.norm = LayerNorm(dmodel)
    self.masked_mha = TPMultiHeadAttention(dmodel,nhead)
    self.norm1 = LayerNorm(dmodel)
    self.enc_dec_attention = TPMultiHeadAttention(dmodel,nhead)
    self.norm2 = LayerNorm(dmodel)
    self.ffn = FFN(dmodel,df)
    self.norm3 = LayerNorm(dmodel)
  
  def forward(self,x,enc,padding_mask,other_mask):
    x = self.norm(x)
    z1= self.masked_mha(x,padding_mask,mask=True)
    z1 = self.norm1(x+z1)
    z2= self.enc_dec_attention(z1,padding_mask,enc=enc,mask=False,other_mask=other_mask)
    z2 = self.norm2(z1+z2)
    y = self.ffn(z2)
    return self.norm3(z2+y)

In [ ]:
class TPTransformer(pl.LightningModule):
    def __init__(self,voc_size,dmodel,df,nhead,nlayers,dropout=0.1):
      super().__init__()
      self.save_hyperparameters()
      self.nlayers = nlayers
      self.embedding = EmbeddingLayer(voc_size,dmodel)
      self.encoders = nn.ModuleList([Encoder(dmodel,df,nhead) for i in range(nlayers)])
      self.decoders = nn.ModuleList([Decoder(dmodel,df,nhead) for i in range(nlayers)])
      #self.drop1 = nn.Dropout(p=dropout)
      #self.drop2 = nn.Dropout(p=dropout)
      self.dmodel = dmodel
      self.metric = nn.CrossEntropyLoss()

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        input, teacher = batch
        in_pad_mask = get_padding_mask(input)
        out_pad_mask = get_padding_mask(teacher)
        x,output = tran(input,teacher,in_pad_mask,out_pad_mask)
        pred = output[:,:-1,:].reshape(output.shape[0]*(output.shape[1] - 1),output.shape[2])
        target = teacher[:,1:].reshape(teacher.shape[0]*(teacher.shape[1] - 1))
        loss = self.metric(pred,target)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(tran.parameters(),lr=1e-04, betas=(0.9, 0.995), eps=1e-09)
        return optimizer
    
    def accuracy(self,pred,targ):
      c = 0
      for i in range(targ.shape[0]):
        if targ[i] == pred[i]:
          c += 1
      return c

    def test_step(self, batch, batch_idx):
        x,y = batch
        enc = True
        trg = y.clone()
        input_pad_mask = get_padding_mask(x)
        for i in range(1,y.shape[1]):
          output_pad_mask = get_padding_mask(trg[:,:i])
          x,output = tran(x,trg[:,:i],input_pad_mask,output_pad_mask,encoding = enc)
          trg[:,i] = output.argmax(-1)[:,-1]
          enc = False
        target_pad_mask = get_padding_mask(y[:,1:])
        acc = self.accuracy(output.argmax(-1)[target_pad_mask[:,0,:]], y[:,1:][target_pad_mask[:,0,:]])
        self.log('test_acc', acc)
        return acc

    def forward(self,x,z,in_padding_mask,out_padding_mask,encoding=True):
      if encoding:
        emb = self.embedding(x)
        t = position_embedding(x.shape[0],x.shape[1],self.dmodel,in_padding_mask)
        x = emb + t
        #x = self.drop1(x)
        for i in range(self.nlayers):
          x = self.encoders[i](x,in_padding_mask)

      emb = self.embedding(z)
      t = position_embedding(z.shape[0],z.shape[1],self.dmodel,out_padding_mask)
      z = emb + t
      #z = self.drop2(z)
      for i in range(self.nlayers):
        z = self.decoders[i](z,x,out_padding_mask,in_padding_mask)
      
      z = z @ self.embedding.E.weight.T

      return x,z

    def predict_step(self, batch, batch_idx):
        x,y = batch
        enc = True
        trg = y.clone()
        input_pad_mask = get_padding_mask(x)
        for i in range(1,y.shape[1]):
          output_pad_mask = get_padding_mask(trg[:,:i])
          x,output = tran(x,trg[:,:i],input_pad_mask,output_pad_mask,encoding = enc)
          trg[:,i] = output.argmax(-1)[:,-1]
          enc = False
        return output.argmax(-1)

In [ ]:
os.environ['WANDB_API_KEY'] = 'ec7e306bca4079bed1f89da133ac2940aa0e531c'

In [ ]:
dmodel = 512
dk,dv = 64,64
nhead = 8
df = 512
nlayers = 6
vocabulary_size = len(vocab)
enable_checkpointing = True
resume_training = True
load_model = True
load_model_dir = "/content/drive/MyDrive/Deep learning/model_checkpoints/TPTransformer/last.ckpt" #change path to load a different checkpoint

In [ ]:
tran = TPTransformer(vocabulary_size,dmodel,df,nhead,nlayers)
if load_model == True:
  tran = TPTransformer.load_from_checkpoint(load_model_dir)

In [ ]:
from pytorch_lightning.loggers import WandbLogger
import wandb
wandb.login()
wandb.init(project='DeepLearning')
wandb.watch(tran,log_freq=10)
wandb_logger = WandbLogger(project='DeepLearning')

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: celestini-1845289. Use `wandb login --relogin` to force relogin


In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(dirpath="/content/drive/MyDrive/Deep learning/model_checkpoints/TPTransformer", save_last =True, save_on_train_epoch_end = True)

#build the correct callbacks
callbacks = [TQDMProgressBar(refresh_rate=20)]
ckpt_path = load_model_dir

#handle checkpointing
if resume_training == False:
  ckpt_path = None
if enable_checkpointing == True:
  callbacks = [checkpoint_callback, TQDMProgressBar(refresh_rate=20)]
  
#passing it to the trainer
gpu = 0
if torch.cuda.is_available() : 
  gpu = 1
trainer = pl.Trainer(enable_checkpointing=enable_checkpointing ,gpus=gpu, max_epochs=2, logger=wandb_logger, callbacks=callbacks)
#trainer.fit(ckpt_path=ckpt_path, model=tran, datamodule=dataset)
print(checkpoint_callback.best_model_path)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:467: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Execute if you want interpolate data:

In [ ]:
trainer.test(tran,datamodule=dataset)

INFO:lightning_fabric.accelerators.cuda:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:232: UserWarning: You called `self.log('test_acc', ...)` in your `test_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                10997.9375
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 10997.9375}]

In [ ]:
wandb.finish()